In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error

In [69]:
file_path="/Users/elenavella/Library/CloudStorage/OneDrive-Personal/masters/Datasets_full/Dataset_full_year/EGLL_2023-04-01_00-00_till_2024-03-31_23-30_cleaned"
df=pd.read_csv(f'{file_path}.csv')
df=df.sort_values(by='TimeStamp')
# Drop unnamed columns
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [70]:
df.columns

Index(['Flight Number', 'Holding Time (minutes)', 'Type Code', 'Time over fix',
       'Big', 'Ock', 'Bov', 'Lam', 'WTC L', 'WTC M', 'WTC H', 'WTC J',
       'Engine Jet', 'Engine Turboprop/shaft', 'TimeStamp', 'Ceiling at -0',
       'Wind at -0', 'Precip at -0', 'Freezing at -0', 'Phenomena at -0',
       'Wind Direction at -0', 'Wind Speed at -0', 'Ceiling at -30',
       'Wind at -30', 'Precip at -30', 'Freezing at -30', 'Phenomena at -30',
       'Wind Direction at -30', 'Wind Speed at -30', 'Ceiling at -60',
       'Wind at -60', 'Precip at -60', 'Freezing at -60', 'Phenomena at -60',
       'Wind Direction at -60', 'Wind Speed at -60', 'day', 'month', 'year',
       'hour', 'minute', 'day_of_week', 'Day_0', 'Day_1', 'Day_2', 'Day_3',
       'Day_4', 'Day_5', 'Day_6', 'Decimal Hours -0', 'Decimal Hours -30',
       'Decimal Hours -60', 'Time leaving fix', 'Holding flights at -0',
       'Holding flights at -30', 'Holding flights at -60', 'Last Timestamp',
       'Last Track', 'La

In [71]:
df

,Flight Number,Holding Time (minutes),Type Code,Time over fix,Big,Ock,Bov,Lam,WTC L,WTC M,...,Altitude at -60,Departures delayIndex at -0,Arrivals numCancelled at -0,Arrivals delayIndex at -0,Departures delayIndex at -30,Arrivals numCancelled at -30,Arrivals delayIndex at -30,Departures delayIndex at -60,Arrivals numCancelled at -60,Arrivals delayIndex at -60
0,VIR450 _4170,0.0,B789,2023-04-01 03:31:29,1,0,0,0,0.0,0.0,...,40000.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
96704,BAW074 _2949,0.0,B77W,2023-04-01 03:58:29,0,1,0,0,0.0,0.0,...,36000.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
96698,BAW124 _1015,0.0,B772,2023-04-01 04:08:59,0,0,0,1,0.0,0.0,...,39000.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
96703,CPA251 _7192,0.0,B77W,2023-04-01 04:11:29,0,0,0,1,0.0,0.0,...,35000.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
96696,BAW58L _1080,0.0,B772,2023-04-01 04:22:29,1,0,0,0,0.0,0.0,...,38000.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185252,BAW49TA _2662,6.5,A320,2024-03-31 21:23:59,0,0,1,0,0.0,1.0,...,33000.0,1.33,0.0,0.0,1.50,1.0,0.0,1.67,1.0,0.0
187858,BAW64VC _2697,7.0,A319,2024-03-31 21:26:59,0,0,1,0,0.0,1.0,...,38000.0,1.33,0.0,0.0,1.50,1.0,0.0,1.67,1.0,0.0
151548,BAW597C _2185,5.0,A320,2024-03-31 21:35:29,0,0,1,0,0.0,1.0,...,38000.0,1.17,0.0,0.0,1.44,1.0,0.0,1.61,1.0,0.0
145101,BAW900A _2227,4.5,A320,2024-03-31 21:31:59,0,0,1,0,0.0,1.0,...,37000.0,1.17,0.0,0.0,1.44,1.0,0.0,1.61,1.0,0.0


In [72]:
def catergory (df,stack_name,stack_identifers):
    # 1) Get the column name with the maximum value for each row
    df[stack_name] = df[stack_identifers].idxmax(axis=1)
    
    # 2) Convert that column to a categorical type
    df[stack_name] = df[stack_name].astype("category")
    
    # Optionally, drop the original one-hot columns
    
    df.drop(stack_identifers, axis=1, inplace=True)
    return df

In [73]:
df=catergory(df,"stack_cat",["Big", "Ock", "Bov", "Lam"])
df=catergory(df,"Engine_cat",['Engine Jet', 'Engine Turboprop/shaft'])
df=catergory(df,"WTC_cat",['WTC L', 'WTC M', 'WTC H', 'WTC J'])
df=catergory(df,"Day_of_the_week_cat",['Day_0', 'Day_1', 'Day_2', 'Day_3', 'Day_4', 'Day_5','Day_6'])
df=catergory(df,"Runway_cat",['Runway 09L', 'Runway 09R', 'Runway 27L','Runway 27R'])



/var/folders/c5/5dvmjdvx2bn_w7l_8s7vlxp40000gn/T/ipykernel_56541/1701355584.py:3: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  df[stack_name] = df[stack_identifers].idxmax(axis=1)


In [74]:
print(df)
print(df.dtypes)

        Flight Number  Holding Time (minutes) Type Code        Time over fix  \
0       VIR450  _4170                     0.0      B789  2023-04-01 03:31:29   
96704   BAW074  _2949                     0.0      B77W  2023-04-01 03:58:29   
96698   BAW124  _1015                     0.0      B772  2023-04-01 04:08:59   
96703   CPA251  _7192                     0.0      B77W  2023-04-01 04:11:29   
96696   BAW58L  _1080                     0.0      B772  2023-04-01 04:22:29   
...               ...                     ...       ...                  ...   
185252  BAW49TA _2662                     6.5      A320  2024-03-31 21:23:59   
187858  BAW64VC _2697                     7.0      A319  2024-03-31 21:26:59   
151548  BAW597C _2185                     5.0      A320  2024-03-31 21:35:29   
145101  BAW900A _2227                     4.5      A320  2024-03-31 21:31:59   
156209  TAP66JC _7201                     5.0      A320  2024-03-31 21:42:59   

                  TimeStamp  Ceiling at

In [75]:
df.to_csv(f'{file_path}_cat.csv')

In [76]:
df_model1 = df[['Flight Number','Holding Time (minutes)','TimeStamp','Type Code','stack_cat' , 'WTC_cat', 'Engine_cat','Ceiling at -0', 'Wind at -0', 'Precip at -0', 'Freezing at -0','Phenomena at -0', 'day','month', 'Day_of_the_week_cat', 'Decimal Hours -0','Holding flights at -0', 'Runway_cat','Crosswind Component at -0', 'Headwind Component at -0','No of Landings 1HR at -0', 'Ground Speed at -0', 'Track at -0','Altitude at -0', 'Departures delayIndex at -0','Arrivals numCancelled at -0', 'Arrivals delayIndex at -0']]


In [78]:
#pip install ydata-profiling
from ydata_profiling import ProfileReport
import webbrowser

# Generate the profile report
profile = ProfileReport(df_model1, title="YData Profiling Report")

# Save the report as an HTML file
report_path = "ydata_profiling_report_cat.html"
profile.to_file(report_path)

# Open the report in the default web browser
webbrowser.open_new_tab(report_path)


/opt/anaconda3/envs/traffic/lib/python3.10/site-packages/ydata_profiling/utils/dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

True

In [82]:
import sweetviz as sv
import pandas as pd



# Generate the Sweetviz report
report = sv.analyze(df_model1)

# Save and open the report
report.show_html("sweetviz_report_cat.html")


                                             |      | [  0%]   00:00 -> (? left)

Report sweetviz_report_cat.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [ ]:
file_path="/Users/elenavella/Library/CloudStorage/OneDrive-Personal/masters/Datasets_full/Dataset_full_year/EGLL_2023-04-01_00-00_till_2024-03-31_23-30_cleaned"
df=pd.read_csv(f'{file_path}.csv')
df=df.sort_values(by='TimeStamp')
# Drop unnamed columns
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]